Crearemos una red neuronal recurrente para clasificar reviews de películas. Las clases son "positivo" o "negativo".
Primero descargamos los datos

In [ ]:
!mkdir data
!wget -c https://www.ivan-sipiran.com/downloads/labels.txt
!wget -c https://www.ivan-sipiran.com/downloads/reviews.txt
!mv *.txt data/

--2025-09-12 00:24:19--  https://www.ivan-sipiran.com/downloads/labels.txt
Resolving www.ivan-sipiran.com (www.ivan-sipiran.com)... 66.96.149.31
Connecting to www.ivan-sipiran.com (www.ivan-sipiran.com)|66.96.149.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K   978KB/s    in 0.2s    

2025-09-12 00:24:20 (978 KB/s) - ‘labels.txt’ saved [225000/225000]

--2025-09-12 00:24:20--  https://www.ivan-sipiran.com/downloads/reviews.txt
Resolving www.ivan-sipiran.com (www.ivan-sipiran.com)... 66.96.149.31
Connecting to www.ivan-sipiran.com (www.ivan-sipiran.com)|66.96.149.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33678267 (32M) [text/plain]
Saving to: ‘reviews.txt’

reviews.txt         100%[===================>]  32.12M  25.0MB/s    in 1.3s    

2025-09-12 00:24:22 (25.0 MB/s) - ‘reviews.txt’ saved [33678267/33678267]



In [ ]:
#Cargamos los archivos de texto
with open('data/reviews.txt', 'r') as f:
  reviews = f.read()
with open('data/labels.txt', 'r') as f:
  labels = f.read()

#reviews y labels como un solo string que almacena todo el archivo

In [ ]:
print(reviews[:200])
print()
print(labels[:18])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  

positive
negative



# Pre-procesamiento de datos

Necesitamos tener los datos en una forma que haga posible el uso de una red nueronal para procesarlo. En este laboratorio, usaremos una representación a nivel de palabras, por lo que necesitamos una forma de representar cada palabra de forma numérica. Aquí aprenderemos cómo procesar el texto y convertirlo en una representación idónea.

Para simplificar el trabajo, tendremos en cuenta:

*   No nos interesan los signos de puntuación
*   Dividir los reviews (un solo string) en strings individuales para cada review. Usaremos el `\n` como delimitador.

Primero, eliminamos los signos de puntuación y dividimos el texto en palabras indiduales



In [ ]:
from string import punctuation

print(punctuation)

reviews = reviews.lower()

all_text = ''.join([c for c in reviews if c not in punctuation])

#all_text[:500]

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
#Removemos los saltos de línea, y juntamos todo el texto de nuevo
reviews_split = all_text.split('\n')
all_text = ''.join(reviews_split)

words = all_text.split()
print(words[:30])

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', 'such', 'as', 'teachers', 'my', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me']


# Codificando las palabras

Vamos a representar cada palabra con un único número entero que representará su índice. Para eso construiremos un diccionario que mapee palabras a números.

In [ ]:
from collections import Counter

counts = Counter(words) #Construye un diccionario de palabras. Las claves son las palabras y los valores son la frecuencia
vocab = sorted(counts, key=counts.get, reverse=True) #Ordenamos la palabras por frecuencia
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)} #Construimos un diccionario para mapear palabra a número entero. Empezamos los índices en 1

#Ahora convertimos cada palabra de los reviews en índices
reviews_ints = []
for review in reviews_split:
  reviews_ints.append([vocab_to_int[word] for word in review.split()])


In [ ]:
#Cada review ahora se representa como una secuencia de números (índices)
print(reviews_split[0])
print(reviews_ints[0])

bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t   
[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 7

In [ ]:
#Cuántas palabras hay en el diccionario?
print('Palabras únicas:', len(vocab_to_int))
print()

Palabras únicas: 74072



#Embedding de etiquetas
Las etiquetas se convierten a números para poder ser ingresadas a la red neuronal.


In [ ]:
import numpy as np

labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split]) # 1 si es positivo, 0 si no

# Longitud de Secuencias
Los reviews tienen distinto tamaño. Para poder procesar estos datos de manera efectiva en una red neuronal, tenemos que uniformizar los tamaños. Esto permite la creación de batches para el entrenamiento.

In [ ]:
#Sacamos algunas estadísticas de los datos
review_lens = Counter([len(x) for x in reviews_ints]) #Contamos cuantas palabras hay en cada review
print("Reviews de longitud cero:", review_lens[0])
print('Máxima longitud:', max(review_lens))

Reviews de longitud cero: 1
Máxima longitud: 2514


Hay reviews vacíos y reviews muy grandes para ser procesados por una RNN. Primero eliminamos los reviews vacíos y truncamos los reviews muy grandes

In [ ]:
print('Reviews antes de eliminación:', len(reviews_ints))

#Extraemos los índices de todos los reviews que tienen longitud > 0
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review)!=0]

#Nos quedamos solo con los reviews con longitud > 0
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]

#Lo mismo con los labels
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Reviews después de eliminación:', len(reviews_ints))

Reviews antes de eliminación: 25001
Reviews después de eliminación: 25000


# Padding
La estrategia para uniformizar el tamaño de las secuencias es como sigue:


*   Definir un tamaño para las secuencias: `seq_length`
*   Reviews con más de `seq_length` palabras, truncamos el review a las primeras `seq_length` palabras.
*   Reviews con menos de `seq_length` palabras, aplicamos padding con 0's al inicio de la secuencia. Por ejemplo el review `['best', 'movie', 'ever']`, `[117, 18, 128]` quedaría como `[0, 0 ,0, ...,117, 18, 128]`

Como cada review ahora tendrá la misma longitud podemos convertir los datos en un array 2D, más conveniente para el proceso posterior.

In [ ]:
def pad_features(reviews_ints, seq_length):
  features = np.zeros((len(reviews_ints), seq_length), dtype=int)

  #Para cada review, se coloca en la matriz
  for i, row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_length]

  return features

In [ ]:
#Probamos el padding
seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

print(features.shape)
print(features[:30,:30])

(25000, 200)
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35  1819    16
    154    19   114     3  1305     5   336   147    22     1   857    12
     70   281  1168   399    36   120]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819    56    17
   4504  5616   140 11725     5   996  4919  2933  4462   566  1201    36
      6  1518    96     3   744     4]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0   

#Partición de los datos


In [ ]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
split_idx = int(len(features)*0.8)
train_x, remaining_x = features[:split_idx], features[split_idx:] # Reviews
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:] # Etiquetas

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:] # 50% test
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:] # 50% validación

## print out the shapes of your resultant feature data
print("\t\t\tFeatures:")
print("Train set: \t\t{}".format(train_x.shape),
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Features:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


#Datasets y Dataloaders


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# crear Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

print("N° interaciones train: ", len(train_loader))
print("N° interaciones validación: ", len(valid_loader))
print("N° interaciones test: ", len(test_loader))

N° interaciones train:  400
N° interaciones validación:  50
N° interaciones test:  50


#Creación de red neuronal
Nuestra red tendrá los siguientes componentes:
1. Una [capa embedding](https://pytorch.org/docs/stable/nn.html#embedding) que convierta tokens de palabras (índices) en embeddings de un tamaño específico.
2. Una [capa LSTM](https://pytorch.org/docs/stable/nn.html#lstm) definida por el tamaño del estado oculto y el número de capas.
3. Una capa de salida fully-connected que mapee la salida del LSTM al tamaño de salida deseado
4. Una capa de activación sigmoide que convierta la salida a valores 0-1; debe retornar **solo la última salida sigmoide** como salida d ela red.

### La capa embedding

Necesitamos agregar una [capa embedding](https://pytorch.org/docs/stable/nn.html#embedding). Una opción podría ser usar one-hot encoding para el vocabulario, pero tenemos demasiadas palabras (~74,000). Así que mejor usamos una capa de embedding que sirva como tabla look-up para una determinada palabra. También se podría usar Word2Vec (embeddings preentrenados), pero ahora usamos una capa embedding que sea aprendida para este problema específico, usándolo como una forma de reducción de la dimensión del vocabulario original.

### La capa LSTM

Crearemos un [LSTM](https://pytorch.org/docs/stable/nn.html#lstm) como red recurrente, con los siguientes parámetros input_size, hidden_dim, el número de capas, la probabilidad de dropout (para dropout entre múltiples), y un parámetro batch_first.



In [ ]:
# Chequear si tenemos GPU
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [ ]:
#Creamos la red neuronal
import torch.nn as nn

class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):

        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # Capas embedding y LSTM
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            dropout=drop_prob, batch_first=True)

        # dropout
        self.dropout = nn.Dropout(drop_prob) # Apaga aleatoriamente 50% de las neuronas

        # Capa lineal y sigmoide
        self.fc = nn.Linear(hidden_dim, output_size) # 256, 1
        self.sig = nn.Sigmoid() # Activación. Toma un número y lo convierte entre 0 y 1

    def forward(self, x, hidden):

        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)

        #Tomamos solo el último valor de salida del LSTM
        lstm_out = lstm_out[:,-1,:]

        # dropout y fully-connected
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # sigmoide
        sig_out = self.sig(out)

        # retornar sigmoide y último estado oculto
        return sig_out, hidden

    # crea/reinicia el estado inicial del LSTM (oculto y memoria), con tamaño (n_layers, batch_size, hidden_dim)
    # lleno de ceros
    def init_hidden(self, batch_size):
        # Crea dos nuevos tensores con tamaño n_layers x batch_size x hidden_dim,
        # inicializados a cero, para estado oculto y memoria de LSTM
        weight = next(self.parameters()).data # Obtiene el tensor de pesos de la red

        if(train_on_gpu):
          hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
          hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden

In [ ]:
# Instanciamos la red
vocab_size = len(vocab_to_int) + 1 # +1 for zero padding + our word tokens
output_size = 1
embedding_dim = 400 # Tamaño del vector de embedding. Arbitrario, no es fijo, depende del problema. 25000 x 400
hidden_dim = 256 # Estado oculto del LSTM. En cada paso del tiempo, el LSTM porduce un vector con 256 valores.
                 # Representa lo que la red ha recordado hasta ese punto de la secuencia.
n_layers = 2 # 2 LSTM

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


# Entrenamiento

Usaremos Binary Cross Entropy Loss, ideal para problemas binarios que usan sigmoide. Usamos gradient clipping también para evitar que los gradientes crezcan mucho.

In [ ]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss() # Binary Cross Entropy Loss
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

Sobre [Exploiding gradients](https://www.geeksforgeeks.org/deep-learning/vanishing-and-exploding-gradients-problems-in-deep-learning/)

In [ ]:
# training params

epochs = 4

counter = 0
print_every = 100
clip=5 # gradient clipping: evita el "Exploiding Gradient"

# Enviar red al GPU
if(train_on_gpu):
    net.cuda()

net.train()
# Bucle de entrenamiento
for e in range(epochs):
    # Inicializar estado oculto
    h = net.init_hidden(batch_size)

    # Bucle para batchs
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Crear nuevas variables para estados ocultos, sino se haría
        # backprop para todos los pasos del bucle
        h = tuple([each.data for each in h])

        net.zero_grad()

        # Hacer pasada forward
        output, h = net(inputs, h)

        # Calcular loss y hacer backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # gradient clipping
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # Mensajes
        if counter % print_every == 0:
            # Validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Época: {}/{}...".format(e+1, epochs),
                  "Paso: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Época: 1/4... Paso: 100... Loss: 0.558549... Val Loss: 0.623495
Época: 1/4... Paso: 200... Loss: 0.706370... Val Loss: 0.691486
Época: 1/4... Paso: 300... Loss: 0.605062... Val Loss: 0.631216
Época: 1/4... Paso: 400... Loss: 0.603836... Val Loss: 0.678208
Época: 2/4... Paso: 500... Loss: 0.531476... Val Loss: 0.567647
Época: 2/4... Paso: 600... Loss: 0.608703... Val Loss: 0.583971
Época: 2/4... Paso: 700... Loss: 0.535602... Val Loss: 0.542013
Época: 2/4... Paso: 800... Loss: 0.422123... Val Loss: 0.578623
Época: 3/4... Paso: 900... Loss: 0.502173... Val Loss: 0.477444
Época: 3/4... Paso: 1000... Loss: 0.346903... Val Loss: 0.498183
Época: 3/4... Paso: 1100... Loss: 0.214732... Val Loss: 0.449111
Época: 3/4... Paso: 1200... Loss: 0.305798... Val Loss: 0.430269
Época: 4/4... Paso: 1300... Loss: 0.262447... Val Loss: 0.524449
Época: 4/4... Paso: 1400... Loss: 0.201503... Val Loss: 0.506572
Época: 4/4... Paso: 1500... Loss: 0.109804... Val Loss: 0.464978
Época: 4/4... Paso: 1600... Loss: 

# Testing

Probamos de dos formas:


*   Calcular accuracy de test
*   Probar inferencia con reviews nuevos



In [ ]:
# Calcular accuracy de test

test_losses = [] # track loss
num_correct = 0

# Iniciar estado oculto
h = net.init_hidden(batch_size)

net.eval()
for inputs, labels in test_loader:

    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()

    output, h = net(inputs, h)

    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    # Convertir probabilidades a clases (0,1)
    pred = torch.round(output.squeeze())

    # Comparar predicciones a labels
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# Accuracy de test
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.451
Test accuracy: 0.819


# Inferencia sobre reviews

In [ ]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower()
    test_text = ''.join([c for c in test_review if c not in punctuation])

    test_words = test_text.split()

    test_ints = []

    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

In [ ]:
def predict(net, test_review, sequence_length=200):

    net.eval()

    test_ints = tokenize_review(test_review)

    seq_length = sequence_length
    features = pad_features(test_ints, seq_length)

    feature_tensor = torch.from_numpy(features)

    batch_size = feature_tensor.size(0)

    h = net.init_hidden(batch_size)

    if(train_on_gpu):
      feature_tensor = feature_tensor.cuda()

    # Aqui ocurre la predicción
    output, h = net(feature_tensor, h)

    pred = torch.round(output.squeeze())
    print('Valor predicho, antes del redondeo: {:.6f}'.format(output.item()))

    # print custom response based on whether test_review is pos/neg
    if(pred.item()==1):
      print('Review positivo!')
    else:
      print('Review negativo!')

In [ ]:
# negative test review
test_review_neg = 'Honestly, the movie sucks.'

# positive test review
test_review_pos = 'Well done. This is the best movie I have ever watched so far.'

In [ ]:
seq_length=200
predict(net, test_review_neg, seq_length)
predict(net, test_review_pos, seq_length)

Valor predicho, antes del redondeo: 0.049095
Review negativo!
Valor predicho, antes del redondeo: 0.959382
Review positivo!
